In [1]:
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
df = pd.read_excel('/home/cody/python_scrap/workout.xlsx')
df = df[['Date','Exercise','Sets']]

In [25]:
class Workout(object):
    
    def __init__(self):
        self.exercises = []
        
    def add_exercise(self,e):
        self.exercises.append(e)
    
    
    def __iter__(self):
        return self.exercises

class Exercise(object):
    
    def __init__(self,rec, name):
        self.name = name.upper()
        self.rec = rec
        self.d = [(int(s.split('*')[0]) , int(s.split('*')[1])) for s in self.rec.split(',')]
        
    def max_weight(self):
        wt = [x[1] for x in self.d]
        return max(wt)
    def __repr__(self):
        return str(self.d)
    def __str__(self):
        return str(self.d)
    def __iter__(self):
        return self.d
    
    
BP = Exercise('12*70,8*90,8*90,8*90,6*100,6*100')
print(BP)

[(12, 70), (8, 90), (8, 90), (8, 90), (6, 100), (6, 100)]


In [26]:
df

,Date,Exercise,Sets
0,2019-02-03,BP,"12*70,8*90,8*90,8*90,6*100,6*100"
1,NaT,DB incline press,"12*70,8*90,8*90,8*90"
2,NaT,Cable pullover,"14*30,8*42.5,8*42.5,8*42.5"
3,NaT,Cable push down,"15*30,10*42.5,8*50,5*50"
4,2019-02-04,lat pull down,"15*35,8*50,8*50,8*50"
5,NaT,Cable row,"15*85,10*100,8*120"
6,NaT,Cable curl,"8*40,8*50,8*50,5*50"
7,NaT,DB curl,"8*50,8*50"
8,2019-02-05,standing dumbbell shoulder press,"15*17.5,10*22.5,8*25,8*27.5"
9,2019-02-07,BP,"12*70,10*90,8*100,8*100,12*100"
